In [47]:
# !pip install synthcity
# !pip install opacus==1.4.0
# !pip install sdmetrics

In [48]:
# !unrar x data.rar

In [49]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from synthcity.plugins import Plugins
from synthcity.plugins.generic.plugin_ddpm import TabDDPMPlugin as DDPM
#from sdv.metadata import SingleTableMetadata
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor
from sdmetrics.reports.single_table import QualityReport
#from sdmetrics.metadata import SingleTableMetadata
import pandas as pd

In [50]:
dir_datasets = '/kaggle/input/nir-exp-1/data/original/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'abalone.csv')
#real_data_1['Rings'] = (real_data_1['Rings'] > 10).astype(int)
real_data_2 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_4 = pd.read_csv(dir_datasets+'my_regression.csv')

# Словарь датасетов для удобства
datasets = {
    # 'abolone': {
    #             "data": real_data_1,
    #             "task": "classification",
    #             "target": "Rings",
    #             "num_columns":
    #             ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
    #             },
    'insurance': {
                    "data": real_data_2,
                    "task": "regression",
                    "target": "expenses",
                    "num_columns":
                    ["age", "bmi", "children"],
                    "cat_columns":
                    ["sex", "smoker", "region"]
                },
    'my_classification': {
                    "data": real_data_3,
                    "task": "classification",
                    "target": "target",
                    "num_columns":
                    ["feature_0", "feature_1", "feature_2", "feature_3", "feature_4"],
                    "cat_columns":
                    []
                },
    'my_regression': {
                    "data": real_data_4,
                    "task": "regression",
                    "target": "target",
                    "num_columns":
                    ["feature_0", "feature_1", "feature_2", "feature_3"],
                    "cat_columns":
                    []
                },
}

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете insurance:
 Количество строк: 1338
 Количество колонок: 7
 Колонки: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses']
 Задача: regression
 Целевая переменная: expenses
 Числовые признаки: ['age', 'bmi', 'children']
 Категориальные признаки: ['sex', 'smoker', 'region']

 Информация о датасете my_classification:
 Количество строк: 900
 Количество колонок: 6
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'target']
 Задача: classification
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
 Категориальные признаки: []

 Информация о датасете my_regression:
 Количество строк: 900
 Количество колонок: 5
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'target']
 Задача: regression
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3']
 Категориальные признаки: []



In [51]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [52]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [53]:
# Определение пространства поиска гиперпараметров для DDPM
ddpm_space = {
    'batch_size': hp.choice('batch_size', [4096]),
    'lr': hp.qloguniform('lr', np.log(3.5e-4), np.log(9.2e-4), 1e-5),
    'num_timesteps': hp.choice('num_timesteps', [1000]),
    'n_layers_hidden': hp.choice('n_layers_hidden', [2, 4, 6]),
    'n_units_hidden': hp.choice('n_units_hidden', [256, 512, 1024]),
    'transformation_num_type': hp.choice('transformation_num_type', ['None'])
}

In [54]:


def evaluate_shape(real_data, synthetic_data, cat_columns):
    """
    Прямое использование метрик SDMetrics без QualityReport.
    Для случаев, когда QualityReport не работает.
    """
    # Обработка входных данных
    if hasattr(synthetic_data, "dataframe"):
        synthetic = synthetic_data.dataframe()
    else:
        synthetic = synthetic_data.copy()

    real_data = pd.get_dummies(real_data, columns=cat_columns, drop_first=True)
    synthetic = pd.get_dummies(synthetic, columns=cat_columns, drop_first=True)
    
    # После get_dummies
    real_data = real_data.astype(float)
    synthetic = synthetic.astype(float)

    # Пытаемся использовать прямые метрики
    shape_scores = []

    from sdmetrics.single_column import KSComplement

    for column in real_data.select_dtypes(include=['number']).columns:
        if column in synthetic.columns:
            ks_score = KSComplement.compute(
                real_data=real_data[column],
                synthetic_data=synthetic[column]
            )
            shape_scores.append(ks_score)

    return sum(shape_scores) / len(shape_scores) if shape_scores else 0.0

In [55]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [56]:
from sklearn.model_selection import KFold

def generate_and_evaluate_shape_scores(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    data = process_data(dataset_info['data'], dataset_info['num_columns'], params['transformation_num_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    shape_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        # Создание и обучение DDPM с заданными параметрами
        ddpm = DDPM(
            lr=params['lr'],
            num_timesteps=params['num_timesteps'],
            model_params = dict(n_layers_hidden=params['n_layers_hidden'],
                                n_units_hidden=params['n_units_hidden'],
                                dropout=0.0),
            batch_size=params['batch_size']
        )

        # Обучение модели на реальных данных
        with suppress_all_output():
            ddpm.fit(train_data)

        # Генерация синтетических данных
        synthetic_data = ddpm.generate(len(train_data))

        # Вычисляем Shape-score
        score = evaluate_shape(
            test_data,
            synthetic_data,
        dataset_info["cat_columns"])
        
        shape_scores.append(score)

    return np.mean(shape_scores)

In [57]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        shape_score = generate_and_evaluate_shape_scores(params, dataset_info)

        print(f"Shape-score: {shape_score}")
        print(params)
        print("="*50)

        # Для Shape-score мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -shape_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'pair_score': shape_score
        }

    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(42)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ddpm_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Получаем лучшие параметры в читаемом виде
    best_params = {
        'batch_size': 4096,
        'num_timesteps': 1000,
        'lr': best['lr'],
        'dropout': 0.0,
        'transformation_num_type': ['None'][best['transformation_num_type']],
        'n_layers_hidden': [2, 4, 6][best['n_layers_hidden']],
        'n_units_hidden': [256, 512, 1024][best['n_units_hidden']],
    }

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_shape_score = best_trial['result'].get('shape_score', None)

    results = {
        'best_params': best_params,
        'best_shape_score': best_shape_score,
        'shape_score_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший Shape-score для {dataset_name}: {best_shape_score}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [58]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: insurance
Shape-score: 0.8991446603554226                       
{'batch_size': 4096, 'lr': 0.00064, 'n_layers_hidden': 6, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_num_type': 'None'}
Shape-score: 0.7351353595748215                                                     
{'batch_size': 4096, 'lr': 0.00063, 'n_layers_hidden': 2, 'n_units_hidden': 1024, 'num_timesteps': 1000, 'transformation_num_type': 'None'}
Shape-score: 0.7680202624148812                                                     
{'batch_size': 4096, 'lr': 0.00038, 'n_layers_hidden': 2, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_num_type': 'None'}
Shape-score: 0.8224962630792226                                                     
{'batch_size': 4096, 'lr': 0.00043000000000000004, 'n_layers_hidden': 4, 'n_units_hidden': 512, 'num_timesteps': 1000, 'transformation_num_type': 'None'}
Shape-score: 0.8391463212091015                                                   

In [59]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_shape_score'])

insurance 0.9618834080717488
my_classification 0.9249074074074072
my_regression 0.9380000000000001
